# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

[/home/runner/work/polars/polars/polars/polars-io/src/ndjson/core.rs:162] &data_type = Struct(
    [
        Field {
            name: "Context_Name",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_MaritalStatus",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_CLV",
            data_type: Int64,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_City",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "IH_Web_Inbound_Accepted_pxLastGroupID",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Decision_Outcome",
            data_type: LargeUtf8,
            is_null

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""../../../../da…","""12859885246854…","""10210477675971…","""10997186788732…","""44106475271901…",false
1.4856e19,"""../../../../da…","""88270735928205…","""10401722668432…","""10997186788732…","""11825334017337…",true
5.6458e17,"""../../../../da…","""22556041647741…","""86241523502647…","""23474888937542…","""44106475271901…",false
4.0723e18,"""../../../../da…","""88270735928205…","""16581677616562…","""11307913782465…","""44106475271901…",false
1.4677e19,"""../../../../da…","""12859885246854…","""17333389543206…","""11307913782465…","""15100937324479…",true
1.6330e19,"""../../../../da…","""22556041647741…","""16283817402730…","""92479318656708…",null,false
8.7675e18,"""../../../../da…","""22556041647741…","""23234530474294…","""11307913782465…","""15100937324479…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""18212436000661…","""80618032172380…","""FirstMortgage3…","""75064312131227…","""Rejected"""
1.4856e19,"""../../../../da…","""10257363877240…","""14614533492591…","""FirstMortgage3…","""14680488816989…","""Accepted"""
5.6458e17,"""../../../../da…","""96314064957557…","""15522568261299…","""MoneyMarketSav…","""75064312131227…","""Rejected"""
4.0723e18,"""../../../../da…","""10257363877240…","""18348867831145…","""BasicChecking""","""75064312131227…","""Rejected"""
1.4677e19,"""../../../../da…","""18212436000661…","""70460660956318…","""BasicChecking""","""12224416220632…","""Accepted"""
1.6330e19,"""../../../../da…","""96314064957557…","""10982914114550…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""96314064957557…","""13267456661512…","""BasicChecking""","""12224416220632…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""14712511671723…","""18367461422860…","""FirstMortgage3…","""27030892693143…","""Rejected"""
1.4856e19,"""../../../../da…","""83860175640471…","""13424345106070…","""FirstMortgage3…","""40483393723346…","""Accepted"""
5.6458e17,"""../../../../da…","""14830231583772…","""62790578226414…","""MoneyMarketSav…","""27030892693143…","""Rejected"""
4.0723e18,"""../../../../da…","""83860175640471…","""17363297789248…","""BasicChecking""","""27030892693143…","""Rejected"""
1.4677e19,"""../../../../da…","""14712511671723…","""21635620450073…","""BasicChecking""","""10083494647397…","""Accepted"""
1.6330e19,"""../../../../da…","""14830231583772…","""14571853845538…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""14830231583772…","""86064598266790…","""BasicChecking""","""10083494647397…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""17023051013583…","""12209408601860…","""FirstMortgage3…","""42576723920408…","""Rejected"""
1.4856e19,"""86828875678988…","""15006652281023…","""FirstMortgage3…","""10985603390378…","""Accepted"""
5.6458e17,"""26284291378445…","""88275505457418…","""MoneyMarketSav…","""42576723920408…","""Rejected"""
4.0723e18,"""86828875678988…","""84351050764338…","""BasicChecking""","""42576723920408…","""Rejected"""
1.4677e19,"""17023051013583…","""77917030040113…","""BasicChecking""","""94550119013658…","""Accepted"""
1.6330e19,"""26284291378445…","""33353126680045…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""26284291378445…","""21441155365260…","""BasicChecking""","""94550119013658…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'